In [1]:
from matplotlib import pylab
from astropy.coordinates import SkyCoord
import integralclient as ic

import time
import numpy as np

from astropy.table import Table, vstack
from astropy.io import fits
import glob
import re
import datetime

failed to get AUTH with env due to: 'K8S_SECRET_INTEGRAL_CLIENT_SECRET'
got AUTH with homefile for integral-limited


In [2]:
cons=1
nrt=1
when_utc="now"

In [3]:
schedule_interval=100
cache_timeout=900

In [4]:
if when_utc == "now":
    get_when_utc=lambda :datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")
    when_utc=get_when_utc()

    get_when_ijd = lambda : float(ic.converttime("UTC",get_when_utc(),"IJD"))
    when_ijd = get_when_ijd()
else:
    get_when_utc=lambda :when_utc
    when_utc=get_when_utc()

    get_when_ijd = lambda : float(ic.converttime("UTC",get_when_utc(),"IJD"))
    when_ijd = get_when_ijd()


2023-05-19 17:24:10 savchenk-NG-VN7-793G-79EG root[1768483] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-19T15:24:10/IJD


In [6]:
import os

arc_root_prefix = None
arc_root_prefixes = ["/mnt/sshfs/isdc/","/"]

for _ in arc_root_prefixes:
    if os.path.exists(_ + "/isdc/arc/rev_3/idx/scw/GNRL-SCWG-GRP-IDX.fits"):
        print("picking this arc root prefix:",arc_root_prefix)
        arc_root_prefix = _
        break
    
if arc_root_prefix is None:
    raise Exception("no archive found!")

print("found prefix", arc_root_prefix)

scwd_cons = Table.read(fits.open(arc_root_prefix+"/isdc/arc/rev_3/idx/scw/GNRL-SCWG-GRP-IDX.fits"))

print("nrt:",arc_root_prefix+"/isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_*")
print("nrt:",glob.glob(arc_root_prefix+"/isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_*")[-1])

scwd_nrt = Table.read(fits.open(sorted(glob.glob(arc_root_prefix+"/isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_*"))[-1])[1])
    


picking this arc root prefix: None
found prefix /mnt/sshfs/isdc/
nrt: /mnt/sshfs/isdc//isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_*
nrt: /mnt/sshfs/isdc//isdc/pvphase/nrt/ops/idx/scw/GNRL-SCWG-GRP-IDX_20230517083026.fits


In [7]:

most_recent_age_cons=when_ijd-np.max(scwd_cons['TSTOP'])
most_recent_age_nrt=when_ijd-np.max(scwd_nrt['TSTOP'])

most_recent_age_cons,most_recent_age_nrt*24


most_recent_age_nrt_hours=most_recent_age_nrt*24
if most_recent_age_nrt_hours<2:
    nrtstatus="ONLINE"
elif most_recent_age_nrt_hours<10:
    nrtstatus="Delayed"
else:    
    nrtstatus="OFFLINE"
print("NRT",nrtstatus)

most_recent_scw_cons = scwd_cons[np.argmax(scwd_cons['TSTOP'])]
most_recent_scw_nrt = scwd_nrt[np.argmax(scwd_nrt['TSTOP'])]
most_recent_scw_nrt
most_recent_scw_cons

sc_nrt=ic.get_sc(str(most_recent_scw_nrt['TSTOP']))

sc_rt=ic.get_sc(str(when_ijd))
sc_rt



NRT ONLINE


{'barytime': -0.2106233846655137,
 'bodies': {'earth': {'body_dec': -50.99,
   'body_distance': 0,
   'body_in_sc': [118.96209798114637, 59.053402267188865],
   'body_ra': 32.00999999999999,
   'body_size': 3.0834782973724897,
   'integral_parallax': 57.29577951308232,
   'separation': 118383.0},
  'moon': {'body_dec': -50.99,
   'body_distance': 369739.05375714647,
   'body_in_sc': [65.84593367792964, -85.35250307468736],
   'body_ra': 32.00999999999999,
   'body_size': 0.30425042981265915,
   'integral_parallax': 20.73579656448591,
   'separation': 327108.0638259236},
  'sun': {'body_dec': -50.99,
   'body_distance': 150595527.0178759,
   'body_in_sc': [60.99703390208115, -95.28415059435409],
   'body_ra': 32.00999999999999,
   'body_size': 0.26478902989889896,
   'integral_parallax': 0.04505752440207181,
   'separation': 150537481.94353393}},
 'd_scx': 118.96209798114637,
 'd_scy': 41.37584253479982,
 'd_scz': 63.26091947062074,
 'dyz_scy': 30.946597732811135,
 'dyz_scz': 59.0534022

In [8]:
current_rev=float(ic.converttime("UTC",when_utc,"REVNUM"))
current_rev

2023-05-19 17:25:56 savchenk-NG-VN7-793G-79EG root[1768483] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-19T15:24:10/REVNUM


2641.0

In [11]:
rtdata_roots=[
    '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/',
    '/rtdata',
]

for realtime_dump_root in rtdata_roots + [ None ]:
    print("probing",realtime_dump_root,"with",glob.glob(realtime_dump_root+"/lcdump-revol-*.csv"))
    if realtime_dump_root and len(glob.glob(realtime_dump_root+"/lcdump-revol-*.csv"))>0:
        print("this",realtime_dump_root)
        break
    
if not realtime_dump_root:
    raise Exception("no realtime archvie found")

try:
    last_fn=sorted([l for l in glob.glob(realtime_dump_root+"/lcdump-revol-*.csv") if 
                   float(re.search("lcdump-revol-(\d{4}).*.csv",l).groups()[0])<current_rev+1])[-1]
    print(last_fn)

    with open(last_fn, 'r') as f:
        lines = f.read().splitlines()
        first_line = lines[0]
        
        first_nonzero_line = None
        for probed_line in lines:
            if probed_line.split()[2]!="0":
                first_nonzero_line = probed_line
                break

            
        last_line = lines[-1]
        
        last_nonzero_line = None
        for probed_line in lines[::-1]:
            if probed_line.split()[2]!="0":
                last_nonzero_line = probed_line
                break

        print("first",first_line)
        print("first non-zero",first_nonzero_line)
        print("last",last_line)
        print("last non-zero",last_nonzero_line)

        rt_ready=True
        rtfirst_ijd=float(first_line.split()[3])
        rtfirst_nonzero_ijd=float(first_nonzero_line.split()[3])
        
        
        rtlast_utc=last_line.split()[1]
        rtlast_ijd=float(last_line.split()[3])
        
        rtlast_nonzero_utc=last_nonzero_line.split()[1]
        rtlast_nonzero_ijd=float(last_nonzero_line.split()[3])
        
        rtlast_acsrate_50ms=float(last_line.split()[2])
        rtlast_ra=float(last_line.split()[4])
        rtlast_dec=float(last_line.split()[5])
        rtlast_earth_separation_kkm=float(last_line.split()[6])
        rtlast_earth_lightraveltime=float(last_line.split()[7]) 
except Exception as e:
    print(repr(e))
    rt_ready=False

probing /mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/ with ['/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2620-pid-16031.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2622-pid-16031.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2639-pid-00044.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2618-pid-16031.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2640-pid-19641.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2640-pid-17629.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2621-pid-16031.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2640-pid-19678.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2641-pid-00044.csv', '/mnt/sshfs/isdc/projects/integral/data/realtime/spiacs/lcdump-revol-2638-pid-00044.csv', '/mnt/sshfs/isdc/projects/inte

In [15]:
if rt_ready:
    rtlast_age=get_when_ijd()-rtlast_ijd    
    rtlast_nonzero_age=get_when_ijd()-rtlast_nonzero_ijd    
    
    print("rt last age", rtlast_age*24*3600, "non-zero", rtlast_nonzero_age*24*3600)

2023-05-19 17:28:49 savchenk-NG-VN7-793G-79EG root[1768483] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-19T15:28:49/IJD
2023-05-19 17:28:50 savchenk-NG-VN7-793G-79EG root[1768483] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-19T15:28:50/IJD


rt last age 55.81209998927079 non-zero 56.81209990871139


In [17]:
if rt_ready:
    rtlast_age_seconds=rtlast_age*24*3600
    if rtlast_age_seconds<120:
        rtstatus="ONLINE"
    elif rtlast_age_seconds<3600:
        rtstatus="Delayed"
    else:    
        rtstatus="OFFLINE"
    rtstatus
else:
    rtstatus="UNAVAILABLE"
    rtlast_utc="unknown"
    rtlast_ijd=float('NaN')
    rtlast_nonzero_utc="unknown"
    rtlast_nonzero_ijd=float('NaN')
    rtlast_ra=float('NaN')
    rtlast_dec=float('NaN')
    rtlast_earth_separation_kkm=float('NaN')
    rtlast_earth_lightraveltime=float('NaN')
    rtlast_age_seconds=float('NaN')
    rtfirst_ijd=float('NaN')

In [19]:

try:
    import prophet
    prophet.arc_root_prefix=arc_root_prefix

    
    prophecy, prophecy_summary = prophet.predict(time=when_utc)
    
    print("prophecy",prophecy)
except Exception as e:
    print(repr(e))
    prophecy = "the future is unclear"
    prophecy_summary = {}

2023-05-19 17:29:17 savchenk-NG-VN7-793G-79EG root[1768483] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-19T15:24:10/REVNUM


arcroot prefix: /mnt/sshfs/isdc/
patt /mnt/sshfs/isdc//isdc/pvphase/nrt/ops/aux/org/2640/TSF_*.INT
all tsfs ['/mnt/sshfs/isdc//isdc/pvphase/nrt/ops/aux/org/2640/TSF_2640_SSSSSSSSSSSSS_0001.INT']
read                       time          name     value  \
0     2023-05-14T23:22:21Z  AOS_CHK_OPEN       NaN   
1     2023-05-14T23:22:21Z          MOUT       NaN   
2     2023-05-14T23:22:21Z          MOUT       NaN   
3     2023-05-14T23:23:19Z          MOUT       NaN   
4     2023-05-14T23:42:21Z            ED  GEBENT02   
...                    ...           ...       ...   
2589  2023-05-17T12:25:53Z          MOUT       NaN   
2590  2023-05-17T13:05:21Z          MOUT       NaN   
2591  2023-05-17T13:05:55Z  E_ECLIP_STOP       NaN   
2592  2023-05-17T13:05:55Z          MOUT       NaN   
2593  2023-05-17T13:34:02Z      PSF_STOP       NaN   

                                                comment  
0           /* 01 Start of post-AOS status check window  
1     "TM start KIRUNA             

In [20]:
prophecy_summary

{'prevrev': {'num': 2640,
  'start': '2023-05-15T02:12:21',
  'stop': '2023-05-17T07:25:48'},
 'thisrev': {'num': 2641,
  'start': '2023-05-17T17:49:17',
  'stop': '2023-05-20T00:23:28'},
 'nextrev': {'num': 2642,
  'start': '2023-05-20T09:39:55',
  'stop': '2023-05-22T16:13:43'},
 'last_data_utc': None,
 'next_data_utc': None,
 'expected_data_status': 'ONLINE'}

In [21]:
template="""
<html>
<head>
</head>
<body>

<table style="border-collapse: separate;border-spacing: 2px;">

<tr>
<td colspan=5 align="center">
<span style="color:blue; font-size:130%; font-weight:bold">
INTEGRAL status
</span>
</td>
</tr>

<tr>
<td colspan=5 style="font-size:80%">Snapshot at {{ when_utc }} UTC</td>
</tr>

<tr>
<td colspan=5>Orbit {{ curent_rev | int }}, {{ (0.001*rtlast_earth_separation_kkm) | round(2) }} Mm to Earth</td>
</tr>


<tr style="background-color:#ADD8E6">
<td align="center"></td>
<td align="center">State/last</td>
<td align="center">Latency</td>
<td align="center">RA</td>
<td align="center">Dec</td>
</tr>


<tr>
<td style="text-align:center" >Real-time</td>
<td style="text-align:center; color:{{ "green" if rtstatus == "ONLINE" else "grey" if rtstatus == "Delayed"  else "red"  }}">{{ rtstatus }}</td>
<td style="text-align:center">{{ rtlast_age_seconds | round(1) }} s</td>
<td style="text-align:center" >{{ rtlast_ra | round(1) }}</td>
<td style="text-align:center" >{{ rtlast_dec | round(1)}}</td>
</tr>

<tr>
<td style="text-align:center">NRT</td>
<td style="text-align:center">{{ last_nrt_scw_id }}</td>
<td style="text-align:center; color:{{ "text" if nrtstatus == "ONLINE" else "#FFA500" if nrtstatus == "Delayed"  else "red"  }}">{{ (most_recent_age_nrt*24) | round(1) }} h</td>
<td style="text-align:center">{{ last_nrt_scw_ra | round(1) }}</td>
<td style="text-align:center">{{ last_nrt_scw_dec | round(1) }}</td>
</tr>
<tr>
<td style="text-align:center">CONS</td>
<td style="text-align:center">{{ last_cons_scw_id }}</td>
<td style="text-align:center">{{ most_recent_age_cons | round(1) }} d</td>
<td style="text-align:center">{{ last_cons_scw_ra| round(1) }}</td>
<td style="text-align:center">{{ last_cons_scw_dec | round(1) }}</td>
</tr>

<tr style="background-color:#ADD8E6">
<td colspan=5 height="5px"></td>
</tr>

<tr>
<td style="text-align:left" colspan=1>
<a href="https://oda-workflows-integral-observation-summary.odahub.io/api/v1.0/get-html/status/history_html">
INTEGRAL status
</a>
</td>
<td colspan=4>
<a align="right" style="text-align:right" href="fetchme?url=http%3A%2F%2Fintegral.esac.esa.int%2Fisocweb%2Fschedule.html%3Faction%3Dschedule">
Schedule
</a>
</tr>

<tr>
<td style="text-align:left" colspan=5>
{{ prophecy }}
</td>
</tr>

</table>

</body>
</html>
"""

from IPython.core.display import display, HTML
from jinja2 import Environment, BaseLoader

rtemplate = Environment(loader=BaseLoader).from_string(template)

data=dict(
    when_utc=when_utc,
    last_cons_scw_id=most_recent_scw_cons['SWID'],
    last_cons_scw_type=most_recent_scw_cons['SW_TYPE'],
    last_cons_scw_ra=most_recent_scw_cons['RA_SCX'],
    last_cons_scw_dec=most_recent_scw_cons['DEC_SCX'],
    last_nrt_scw_id=most_recent_scw_nrt['SWID'],
    last_nrt_scw_type=most_recent_scw_nrt['SW_TYPE'],
    last_nrt_scw_ra=most_recent_scw_nrt['RA_SCX'], 
    last_nrt_scw_dec=most_recent_scw_nrt['DEC_SCX'],
    earth_separation_kkm=sc_rt['bodies']['earth']['separation'],
    earth_size_deg=sc_rt['bodies']['earth']['body_size'],
    rtlast_utc=rtlast_utc,
    rtlast_ijd=rtlast_ijd,
    rtlast_nonzero_utc=rtlast_nonzero_utc,
    rtlast_nonzero_ijd=rtlast_nonzero_ijd,
    rtlast_ra=sc_rt['scx']['ra'],
    rtlast_dec=sc_rt['scx']['dec'],
    rtlast_earth_separation_kkm=rtlast_earth_separation_kkm,
    rtlast_earth_lightraveltime=rtlast_earth_lightraveltime,
    rtlast_age_seconds=rtlast_age_seconds,
    rtfirst_ijd=rtfirst_ijd,
    most_recent_age_nrt=most_recent_age_nrt,
    most_recent_age_cons=most_recent_age_cons,
    curent_rev=current_rev,
    rtstatus=rtstatus,
    nrtstatus=nrtstatus,
    now=datetime.datetime.now(),
    prophecy=prophecy,
    prophecy_summary=prophecy_summary,
)
data_html = rtemplate.render(**data)

open("status_card.html","w").write(data_html)

display(HTML(data_html))

/tmp/ipykernel_1768483/1870517147.py:86: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [22]:
prophecy_summary

{'prevrev': {'num': 2640,
  'start': '2023-05-15T02:12:21',
  'stop': '2023-05-17T07:25:48'},
 'thisrev': {'num': 2641,
  'start': '2023-05-17T17:49:17',
  'stop': '2023-05-20T00:23:28'},
 'nextrev': {'num': 2642,
  'start': '2023-05-20T09:39:55',
  'stop': '2023-05-22T16:13:43'},
 'last_data_utc': None,
 'next_data_utc': None,
 'expected_data_status': 'ONLINE'}

In [23]:
history_html="status_card.html" # smth
data=data